# Calculating rest-frame photometric properties

While SED fitting offers a fantastic avenue to explore galaxy properties, there are often significant assumptions required. For example, the IMF, SFH, metallicity, dust content/geometry, all must be assumed even though we do not have the appropriate photometric data to properly constrain them at this point. This may lead to inaccurate measurements of galaxy properties, and the associated errors are nearly always underestimated.

As seen in Austin et al 2024 and Cullen et al 2024, UV continuum slope measurements of blue SFGs identified by JWST are biased red by SED fitting, making it more appropriate to measure these directly from the photometric data without the assumptions that are required by the template inputs to Bayesian SED fitting.

In this notebook, we will demonstrate the available `Rest_Frame_Property_Calculator` classes that are available in galfind. These calculate galaxy properties in the rest-frame UV and optical directly from the photometric data and can be applied on either a `Catalogue`, `Galaxy`, or `Photometry_rest` level.

Of course, and as always, we start by instantiating our JOF NIRCam v11 `Catalogue` object. This time we only need to load in the SED results of interest; we use the redshift free EAZY run as an example.

In [1]:
import astropy.units as u
from copy import deepcopy
from galfind import Catalogue, EAZY
from galfind.Data import morgan_version_to_dir

Reading GALFIND config file from: /nvme/scratch/work/austind/GALFIND/galfind/../configs/galfind_config.ini
"Important:  Gaia archive will be intermittently unavailable due to scheduled maintenance on 10-12-2024 from 08:00 to 10:00 (CET)"


Failed to `import dust_attenuation`
Install from the repo with $ pip install git+https://github.com/karllark/dust_attenuation.git


In [2]:
survey = "JOF"
version = "v11"
instrument_names = ["NIRCam"]
aper_diams = [0.32] * u.arcsec
forced_phot_band = ["F277W", "F356W", "F444W"]
min_flux_pc_err = 10.

JOF_cat = Catalogue.pipeline(
    survey,
    version,
    instrument_names = instrument_names, 
    version_to_dir_dict = morgan_version_to_dir,
    aper_diams = aper_diams,
    forced_phot_band = forced_phot_band,
    min_flux_pc_err = min_flux_pc_err
)

SED_fit_params_arr = [{"templates": "fsps_larson", "lowz_zmax": None}]
for SED_fit_params in SED_fit_params_arr:
    EAZY_fitter = EAZY(SED_fit_params)
    EAZY_fitter(JOF_cat, aper_diams[0], load_PDFs = True, load_SEDs = True, update = True)
SED_fit_label = EAZY_fitter.label

INFO:galfind:Loaded aper_diams=<Quantity [0.32] arcsec> for F277W+F356W+F444W
INFO:galfind:Combined mask for NIRCam/F277W+F356W+F444W already exists at /raid/scratch/work/austind/GALFIND_WORK/Masks/JOF/combined/JOF_F277W+F356W+F444W_auto.fits
Calculating depths:   0%|          | 0/15 [00:00<?, ?it/s]
INFO:galfind:Calculated/loaded depths for JOF v11 NIRCam
INFO:galfind:Local depth columns already exist in /raid/scratch/work/austind/GALFIND_WORK/Catalogues/v11/NIRCam/JOF/(0.32)as/JOF_MASTER_Sel-F277W+F356W+F444W_v11.fits
INFO:galfind:Loaded 'has_data_mask' from /raid/scratch/work/austind/GALFIND_WORK/Masks/JOF/has_data_mask/JOF_MASTER_Sel-F277W+F356W+F444W_v11.h5
INFO:galfind:Making JOF v11 JOF_MASTER_Sel-F277W+F356W+F444W_v11 catalogue!
INFO:galfind:Made /raid/scratch/work/austind/GALFIND_WORK/Catalogues/v11/NIRCam/JOF/(0.32)as/JOF_MASTER_Sel-F277W+F356W+F444W_v11.fits catalogue!
INFO:galfind:Making .in file for EAZY_fsps_larson_zfree SED fitting for JOF v11 NIRCam
INFO:galfind:Made .i

Running SED fitting took 0.1s


INFO:galfind:Loading EAZY_fsps_larson property PDFs into JOF v11 NIRCam


Loading properties and associated errors took 1.4s


Constructing redshift PDFs: 100%|██████████| 16335/16335 [00:00<00:00, 30159.02it/s]
INFO:galfind:Finished loading EAZY_fsps_larson property PDFs into JOF v11 NIRCam
INFO:galfind:Loading EAZY_fsps_larson SEDs into JOF v11 NIRCam
Constructing SEDs: 100%|██████████| 16335/16335 [00:01<00:00, 13781.28it/s]
INFO:galfind:Finished loading EAZY_fsps_larson SEDs into JOF v11 NIRCam
INFO:galfind:Updating SED results in galfind catalogue object
Updating galaxy SED results: 100%|██████████| 16335/16335 [00:00<00:00, 155779.74it/s]


## Example 1: Photometry_rest, Galaxy, and Catalogue level calculations

In this example, we will work through the calculation of these properties excluding any associated errors. The main advantage of this is that it will quickly produce results for large surveys or simulations to, for example, determine sample completeness/contamination. Firstly, we need an example `Rest_Frame_Property_Calculator` object; we will create an `UV_Beta_Calculator` object in this instance, although in principle any of the other `Rest_Frame_Property_Calculator` child classes will do the trick. The `UV_Beta_Calculator` class is a subclass of `Rest_Frame_Property_Calculator` and, as with all `Rest_Frame_Property_Calculator` objects, must be instantiated with an explicit `aper_diam` and `SED_fit_label`. The `rest_UV_wav_lims` given here are the default values.

In [3]:
from galfind import UV_Beta_Calculator
beta_calculator = UV_Beta_Calculator(
    aper_diam = aper_diams[0],
    SED_fit_label = SED_fit_label,
    rest_UV_wav_lims = [1_250., 3_000.] * u.AA
)

Calling this object while inserting either a `Photometry_rest`, `Galaxy`, or `Catalogue` as the first argument will perform the calculation. Let's first create a `Photometry_rest` object corresponding to the z=14.63 photometric candidate from Robertson et al. 2023.

In [4]:
phot_rest_z14 = deepcopy(JOF_cat[717].aper_phot[aper_diams[0]].SED_results[SED_fit_label].phot_rest)
print(phot_rest_z14)

****************************************
PHOTOMETRY_REST: z = 14.662217140197754
----------
****************************************



In this example, we will run with `n_chains=1` (i.e. without the Monte Carlo). In the codeblock below this `n_chains` parameter is the only argument that is changed from the default value, which is 10,000.

In [5]:
beta_calculator(
    phot_rest_z14,
    n_chains = 1, 
    output = False,
    overwrite = False,
    n_jobs = 1
)

Using the overloaded __str__ operator, we can see the impact on the properties stored in the `phot_rest_z14` object.

In [6]:
print(phot_rest_z14)
print(phot_rest_z14.properties)

****************************************
PHOTOMETRY_REST: z = 14.662217140197754
----------
****************************************

{'beta_[1250,3000]AA': <Quantity -2.07955873>}


Now let's pass the entire galaxy through and see what happens.

In [7]:
gal_z14 = deepcopy(JOF_cat[717])
print(gal_z14)
beta_calculator(
    gal_z14,
    n_chains = 1
)
print(gal_z14)

****************************************
Galaxy(718, [53.10763,-27.86013]deg)
****************************************
PHOTOMETRY:
----------
Photometry_obs(NIRCam, 0.32 arcsec, EAZY_fsps_larson_zfree)
----------
SELECTION FLAGS:
----------
bluewards_Lya_SNR<2.0_EAZY_fsps_larson_zfree_0.32as: True
redwards_Lya_SNR>5.0,5.0_widebands_EAZY_fsps_larson_zfree_0.32as: True
ALL_redwards_Lya_SNR>2.0_EAZY_fsps_larson_zfree_0.32as: True
red_chi_sq<3.0_EAZY_fsps_larson_zfree_0.32as: True
chi_sq_diff>4.0,dz>0.5_EAZY_fsps_larson_zfree_0.32as: True
zPDF>60%,|dz|/z<0.1_EAZY_fsps_larson_zfree_0.32as: True
unmasked_F090W: True
unmasked_F115W: True
unmasked_F150W: True
unmasked_F162M: True
unmasked_F182M: True
unmasked_F200W: True
unmasked_F210M: True
unmasked_F250M: True
unmasked_F277W: True
unmasked_F300M: True
unmasked_F335M: True
unmasked_F356W: True
unmasked_F410M: True
unmasked_F444W: True
unmasked_NIRCam: True
bluest_band_SNR<2.0_0.32as: True
sex_Re_F277W>45.0mas: True
sex_Re_F356W>45.0mas: True


Of course we can loop through these galaxies to update the catalogue, or we can simply pass the entire catalogue in to calculate these properties.

In [8]:
print(JOF_cat)
JOF_cat_copy = deepcopy(JOF_cat)
beta_calculator(
    JOF_cat_copy,
    n_chains = 1
)
print(JOF_cat_copy)

****************************************
CATALOGUE(JOF,v11,NIRCam):
----------
CAT PATH = /raid/scratch/work/austind/GALFIND_WORK/Catalogues/v11/NIRCam/JOF/(0.32)as/JOF_MASTER_Sel-F277W+F356W+F444W_v11.fits
TOTAL GALAXIES = 16335
RA RANGE = [53.01070689 53.11059594] deg
DEC RANGE = [-27.91226173 -27.83206063] deg
----------
****************************************
MULTIPLE_FILTER
----------
FACILITY: JWST
INSTRUMENT: NIRCam
FILTERS: ['F090W', 'F115W', 'F150W', 'F162M', 'F182M', 'F200W', 'F210M', 'F250M', 'F277W', 'F300M', 'F335M', 'F356W', 'F410M', 'F444W']
****************************************



Calculating beta_[1250,3000]AA: 100%|██████████| 16335/16335 [00:31<00:00, 510.85it/s]


****************************************
CATALOGUE(JOF,v11,NIRCam):
----------
CAT PATH = /raid/scratch/work/austind/GALFIND_WORK/Catalogues/v11/NIRCam/JOF/(0.32)as/JOF_MASTER_Sel-F277W+F356W+F444W_v11.fits
TOTAL GALAXIES = 16335
RA RANGE = [53.01070689 53.11059594] deg
DEC RANGE = [-27.91226173 -27.83206063] deg
----------
****************************************
MULTIPLE_FILTER
----------
FACILITY: JWST
INSTRUMENT: NIRCam
FILTERS: ['F090W', 'F115W', 'F150W', 'F162M', 'F182M', 'F200W', 'F210M', 'F250M', 'F277W', 'F300M', 'F335M', 'F356W', 'F410M', 'F444W']
****************************************



Fantastic! We have now computed the UV beta slopes for every galaxy in our JOF catalogue. Some, of course, will fail and produce NaN's, namely when there are fewer than 2 photometric filters entirely within the rest frame UV wavelength range used. This will mean that depending on the filterset used, some redshift ranges will have more/less precise UV continuum slope measurements, and in some ranges it will be impossible to calculate this quantity.

## Example 2: Running Monte Carlo to produce PDFs

Instead of a single chain, as done in the previous example, we will now compute the UV continuum slope using a Monte Carlo technique to incorporate the photometric errors. We scatter each photometric data point about its error profile, which is assumed to be Gaussian, before re-computing the beta slope for each scattered photometry. This creates an array of beta slopes from which we extract the median, and upper and lower 1σ errors.

In the below codeblock, we calculate this Monte Carlo using 10,000 chains, which is the default.

In [9]:
beta_calculator(
    JOF_cat,
    n_chains = 10_000
)
print(JOF_cat)

Calculating beta_[1250,3000]AA:   0%|          | 0/16335 [00:00<?, ?it/s]

Calculating beta_[1250,3000]AA: 100%|██████████| 16335/16335 [00:17<00:00, 908.01it/s]


****************************************
CATALOGUE(JOF,v11,NIRCam):
----------
CAT PATH = /raid/scratch/work/austind/GALFIND_WORK/Catalogues/v11/NIRCam/JOF/(0.32)as/JOF_MASTER_Sel-F277W+F356W+F444W_v11.fits
TOTAL GALAXIES = 16335
RA RANGE = [53.01070689 53.11059594] deg
DEC RANGE = [-27.91226173 -27.83206063] deg
----------
****************************************
MULTIPLE_FILTER
----------
FACILITY: JWST
INSTRUMENT: NIRCam
FILTERS: ['F090W', 'F115W', 'F150W', 'F162M', 'F182M', 'F200W', 'F210M', 'F250M', 'F277W', 'F300M', 'F335M', 'F356W', 'F410M', 'F444W']
****************************************



In [10]:
# TODO: Showing where the PDFs are stored, and a little about the stored PDF objects/kwargs etc

## Example 3: Rest-frame UV continuum properties

In examples 1 and 2 we have looked at calculating UV continuum slopes only, however galfind also has the ability to calculate many more rest frame properties. In this example, we will go through all calculable properties in the rest-frame UV.

In [11]:
from galfind import (
    mUV_Calculator,
    MUV_Calculator,
    UV_Dust_Attenuation_Calculator,
    LUV_Calculator,
    SFR_UV_Calculator,
    M99, # dust conversion from beta
)

In [12]:
AUV_calculator = UV_Dust_Attenuation_Calculator(
    aper_diam = aper_diams[0],
    SED_fit_label = SED_fit_label,
    rest_UV_wav_lims = [1_250.0, 3_000.0] * u.AA,
    beta_dust_conv = M99,
    ref_wav = 1_500.0 * u.AA
)
AUV_calculator(JOF_cat)
print(JOF_cat)

Calculating beta_[1250,3000]AA:   0%|          | 0/16335 [00:00<?, ?it/s]

Calculating A1500_M99_[1250,3000]AA: 100%|██████████| 16335/16335 [00:25<00:00, 651.39it/s] 


****************************************
CATALOGUE(JOF,v11,NIRCam):
----------
CAT PATH = /raid/scratch/work/austind/GALFIND_WORK/Catalogues/v11/NIRCam/JOF/(0.32)as/JOF_MASTER_Sel-F277W+F356W+F444W_v11.fits
TOTAL GALAXIES = 16335
RA RANGE = [53.01070689 53.11059594] deg
DEC RANGE = [-27.91226173 -27.83206063] deg
----------
****************************************
MULTIPLE_FILTER
----------
FACILITY: JWST
INSTRUMENT: NIRCam
FILTERS: ['F090W', 'F115W', 'F150W', 'F162M', 'F182M', 'F200W', 'F210M', 'F250M', 'F277W', 'F300M', 'F335M', 'F356W', 'F410M', 'F444W']
****************************************



In [13]:
mUV_calculator = mUV_Calculator(
    aper_diam = aper_diams[0],
    SED_fit_label = SED_fit_label,
    rest_UV_wav_lims = [1_250., 3_000.] * u.AA,
    ref_wav = 1_500.0 * u.AA,
    top_hat_width = 100.0 * u.AA,
    resolution = 1.0 * u.AA
)
mUV_calculator(JOF_cat)
print(JOF_cat)

Calculating beta_[1250,3000]AA:   0%|          | 0/16335 [00:00<?, ?it/s]

Calculating m1500_[1250,3000]AA: 100%|██████████| 16335/16335 [00:10<00:00, 1493.05it/s]


****************************************
CATALOGUE(JOF,v11,NIRCam):
----------
CAT PATH = /raid/scratch/work/austind/GALFIND_WORK/Catalogues/v11/NIRCam/JOF/(0.32)as/JOF_MASTER_Sel-F277W+F356W+F444W_v11.fits
TOTAL GALAXIES = 16335
RA RANGE = [53.01070689 53.11059594] deg
DEC RANGE = [-27.91226173 -27.83206063] deg
----------
****************************************
MULTIPLE_FILTER
----------
FACILITY: JWST
INSTRUMENT: NIRCam
FILTERS: ['F090W', 'F115W', 'F150W', 'F162M', 'F182M', 'F200W', 'F210M', 'F250M', 'F277W', 'F300M', 'F335M', 'F356W', 'F410M', 'F444W']
****************************************



In [14]:
MUV_calculator = MUV_Calculator(
    aper_diam = aper_diams[0],
    SED_fit_label = SED_fit_label,
    rest_UV_wav_lims = [1_250., 3_000.] * u.AA,
    ref_wav = 1_500.0 * u.AA,
    top_hat_width = 100.0 * u.AA,
    resolution = 1.0 * u.AA
)
MUV_calculator(JOF_cat)
print(JOF_cat)

Calculating beta_[1250,3000]AA:   0%|          | 0/16335 [00:00<?, ?it/s]

Calculating M1500_[1250,3000]AA: 100%|██████████| 16335/16335 [00:40<00:00, 400.12it/s]


****************************************
CATALOGUE(JOF,v11,NIRCam):
----------
CAT PATH = /raid/scratch/work/austind/GALFIND_WORK/Catalogues/v11/NIRCam/JOF/(0.32)as/JOF_MASTER_Sel-F277W+F356W+F444W_v11.fits
TOTAL GALAXIES = 16335
RA RANGE = [53.01070689 53.11059594] deg
DEC RANGE = [-27.91226173 -27.83206063] deg
----------
****************************************
MULTIPLE_FILTER
----------
FACILITY: JWST
INSTRUMENT: NIRCam
FILTERS: ['F090W', 'F115W', 'F150W', 'F162M', 'F182M', 'F200W', 'F210M', 'F250M', 'F277W', 'F300M', 'F335M', 'F356W', 'F410M', 'F444W']
****************************************



In [15]:
frame = "obs"
LUV_calculator = LUV_Calculator(  
    aper_diam = aper_diams[0],
    SED_fit_label = SED_fit_label,
    frame = frame,
    rest_UV_wav_lims = [1_250.0, 3_000.0] * u.AA,
    ref_wav = 1_500.0 * u.AA,
    beta_dust_conv = M99,
    top_hat_width = 100.0 * u.AA,
    resolution = 1.0 * u.AA
)
LUV_calculator(JOF_cat)
print(JOF_cat)

Calculating L1500_obs_M99dust_[1250,3000]AA: 100%|██████████| 16335/16335 [00:41<00:00, 391.44it/s]


****************************************
CATALOGUE(JOF,v11,NIRCam):
----------
CAT PATH = /raid/scratch/work/austind/GALFIND_WORK/Catalogues/v11/NIRCam/JOF/(0.32)as/JOF_MASTER_Sel-F277W+F356W+F444W_v11.fits
TOTAL GALAXIES = 16335
RA RANGE = [53.01070689 53.11059594] deg
DEC RANGE = [-27.91226173 -27.83206063] deg
----------
****************************************
MULTIPLE_FILTER
----------
FACILITY: JWST
INSTRUMENT: NIRCam
FILTERS: ['F090W', 'F115W', 'F150W', 'F162M', 'F182M', 'F200W', 'F210M', 'F250M', 'F277W', 'F300M', 'F335M', 'F356W', 'F410M', 'F444W']
****************************************



In [16]:
sfr_100myr_calculator = SFR_UV_Calculator(
    aper_diam = aper_diams[0],
    SED_fit_label = SED_fit_label,
    rest_UV_wav_lims = [1_250.0, 3_000.0] * u.AA,
    ref_wav = 1_500.0 * u.AA,
    beta_dust_conv = M99,
    SFR_conv = "MD14",
    top_hat_width = 100.0 * u.AA,
    resolution = 1.0 * u.AA
)
sfr_100myr_calculator(JOF_cat)
print(JOF_cat)

Calculating beta_[1250,3000]AA:   0%|          | 0/16335 [00:00<?, ?it/s]

Calculating SFR1500_M99dust_[1250,3000]AA_MD14: 100%|██████████| 16335/16335 [00:53<00:00, 304.77it/s]


****************************************
CATALOGUE(JOF,v11,NIRCam):
----------
CAT PATH = /raid/scratch/work/austind/GALFIND_WORK/Catalogues/v11/NIRCam/JOF/(0.32)as/JOF_MASTER_Sel-F277W+F356W+F444W_v11.fits
TOTAL GALAXIES = 16335
RA RANGE = [53.01070689 53.11059594] deg
DEC RANGE = [-27.91226173 -27.83206063] deg
----------
****************************************
MULTIPLE_FILTER
----------
FACILITY: JWST
INSTRUMENT: NIRCam
FILTERS: ['F090W', 'F115W', 'F150W', 'F162M', 'F182M', 'F200W', 'F210M', 'F250M', 'F277W', 'F300M', 'F335M', 'F356W', 'F410M', 'F444W']
****************************************



In [18]:
from galfind import Fesc_From_Beta_Calculator
fesc_calculator = Fesc_From_Beta_Calculator(
    aper_diam = aper_diams[0],
    SED_fit_label = SED_fit_label,
    rest_UV_wav_lims = [1_250.0, 3_000.0] * u.AA,
    fesc_conv = "Chisholm22"
)
fesc_calculator(JOF_cat)
print(JOF_cat)

Calculating beta_[1250,3000]AA:   0%|          | 0/16335 [00:00<?, ?it/s]

Calculating fesc=Chisholm22_[1250,3000]AA: 100%|██████████| 16335/16335 [00:01<00:00, 14738.51it/s]


****************************************
CATALOGUE(JOF,v11,NIRCam):
----------
CAT PATH = /raid/scratch/work/austind/GALFIND_WORK/Catalogues/v11/NIRCam/JOF/(0.32)as/JOF_MASTER_Sel-F277W+F356W+F444W_v11.fits
TOTAL GALAXIES = 16335
RA RANGE = [53.01070689 53.11059594] deg
DEC RANGE = [-27.91226173 -27.83206063] deg
----------
****************************************
MULTIPLE_FILTER
----------
FACILITY: JWST
INSTRUMENT: NIRCam
FILTERS: ['F090W', 'F115W', 'F150W', 'F162M', 'F182M', 'F200W', 'F210M', 'F250M', 'F277W', 'F300M', 'F335M', 'F356W', 'F410M', 'F444W']
****************************************



## Example 4: Rest-frame optical emission lines

## Example 5: Dust measurements

## Example 6: Multi-processing with joblib for faster computation

## Example 7: Custom rest-frame property calculators